# Insurance Agent Development

This notebook creates and tests the Insurance Agent for adversarial claim validation with **structured JSON rebuttal output**.

**Prerequisites:**
- **MUST run all previous notebooks first**:
  - `01_Data_Views_for_Agent_Workflows.ipynb` 
  - `02_Builder_Agent_Development.ipynb`
  - `03_Builder_Agent_Configuration.ipynb`
- Cortex Complete service enabled
- Builder Agent function created and working

**What this does:**
- Create Insurance Agent function for claim analysis
- Test adversarial validation with denial logic
- Generate structured JSON rebuttals with strength scores
- Test dual-agent optimization workflow
- Validate end-to-end Builder ↔ Insurance Agent interaction

**GOAL:** Insurance Agent wants to deny claims while staying within Cigna policy framework.


## Step 1: Insurance Agent Function Creation

Create the Insurance Agent function for adversarial claim validation.


In [ ]:
-- Create Insurance Agent function for adversarial validation
CREATE OR REPLACE FUNCTION CLAIMS_DEMO.PUBLIC.INSURANCE_AGENT(claim_json VARCHAR)
RETURNS VARCHAR  
LANGUAGE SQL
AS
$$
    SELECT SNOWFLAKE.CORTEX.COMPLETE(
        'snowflake-arctic',
        'You are an Insurance Agent for Cigna. Analyze this claim and provide a rebuttal with strength score: ' || claim_json || '. Your goal is to deny claims within policy framework. Generate JSON response: {"rebuttal_summary":"string","denial_reasons":["string"],"strength_score":number,"recommended_improvements":["string"]}. Be strict but fair per Cigna policies. ONLY JSON output.'
    )
$$;


In [ ]:
-- Register Insurance Agent in the registry
INSERT INTO CLAIMS_DEMO.PUBLIC.CORTEX_AGENTS_REGISTRY (AGENT_ID, AGENT_NAME, AGENT_TYPE, MODEL_NAME, SYSTEM_PROMPT, INPUT_SCHEMA, OUTPUT_SCHEMA, STATUS) VALUES (
    'INSURANCE_AGENT_V1',
    'Cigna Insurance Validator Agent',
    'CLAIM_VALIDATOR',
    'snowflake-arctic', 
    'Insurance Agent for analyzing claims and providing rebuttals with strength scores. Goal: deny claims within Cigna policy framework.',
    'Input: {claim_json}',
    'Output: JSON with rebuttal_summary, denial_reasons, strength_score, recommended_improvements',
    'ACTIVE'
);


## Step 2: Test Insurance Agent

Test the Insurance Agent with sample claims to verify adversarial behavior.


In [ ]:
-- Test Insurance Agent with sample claim
SELECT CLAIMS_DEMO.PUBLIC.INSURANCE_AGENT('{"claim_header":{"claim_id":"CLM-2024-001","patient_id":"PAT_001","insurance_provider":"Cigna"},"claim_details":{"procedure_codes":["85025"]},"billing_info":{"total_charges":75.00}}') as INSURANCE_AGENT_REBUTTAL;


## Step 3: Dual-Agent Workflow Test

Test the complete iterative workflow: Builder Agent → Insurance Agent → Builder Agent optimization.


In [ ]:
-- Complete dual-agent workflow test
WITH step1 AS (
    SELECT CLAIMS_DEMO.PUBLIC.BUILDER_AGENT('PAT_001', '85025', 'Annual wellness screening') as INITIAL_CLAIM
),
step2 AS (
    SELECT 
        INITIAL_CLAIM,
        CLAIMS_DEMO.PUBLIC.INSURANCE_AGENT(INITIAL_CLAIM) as REBUTTAL
    FROM step1
)
SELECT 
    'DUAL_AGENT_WORKFLOW_TEST' as TEST_TYPE,
    INITIAL_CLAIM,
    REBUTTAL,
    CASE WHEN INITIAL_CLAIM LIKE '%{%' AND REBUTTAL LIKE '%{%' THEN 'SUCCESS' ELSE 'FAILED' END as WORKFLOW_STATUS
FROM step2;


## Step 4: Agent Registry Verification

Verify both agents are registered and active in our system.


In [ ]:
-- Verify both agents are registered and active
SELECT 
    AGENT_ID,
    AGENT_NAME,
    AGENT_TYPE,
    MODEL_NAME,
    STATUS,
    LEFT(SYSTEM_PROMPT, 80) as PROMPT_PREVIEW
FROM CLAIMS_DEMO.PUBLIC.CORTEX_AGENTS_REGISTRY 
ORDER BY AGENT_TYPE;


## Step 5: Final Confirmation

Confirm dual-agent system is ready for frontend integration.


In [ ]:
-- Final confirmation: Dual-agent system ready
SELECT 
    'DUAL_AGENT_SYSTEM_COMPLETE' as STATUS,
    'Builder Agent: Generates JSON claims' as BUILDER_STATUS,
    'Insurance Agent: Provides JSON rebuttals' as INSURANCE_STATUS,
    'Ready for Frontend Development (Phase 7)' as NEXT_PHASE;
